# [Module 0.1] Forecast 입력될 데이타 포맷 작성 (Step by Step)

"Store Item Sales" Data는 Target Time Series 데이터만이 존재 합니다. <br>
이 노트북은 실제 Forecast의 Data Set에 들어갈 포맷을 만드는 작업을 순차적으로 학인 합니다.<br>
이 노트북의 과정은 다음 노트북인 "1.Prepare_Data_File.ipynb"에 축약되어서 실행이 됩니다.

---
또한 데이타 내용을 파악하기 위해서 데이터 탐색 노트북인 "0.0.Option-Exploratory-Data-Analysis.ipynb" 를 실행 하시고 진행 하시는 것을 권고 합니다.



## 데이타 로딩 및 내용 확인

In [1]:
import pandas as pd
import numpy as np
import time
import warnings
import os

In [2]:
data_dir = 'data'
data_file_name = 'train.csv'

In [3]:
df = pd.read_csv(os.path.join(data_dir,data_file_name))
df.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


데이타가 결측이 있는지를 확인 합니다.

In [4]:
df.isnull().sum()

date     0
store    0
item     0
sales    0
dtype: int64

## Make a format for Forecast

Amazon Forecast에 입력이 될 포맷으로 변경 합니다. <br>


### item 컬럼 이름 변경
아래는 item --> item_id 이름으로 변경 합니다.

In [5]:
data_df = df.copy()
data_df = data_df.rename(columns={'item':'item_id'})

data_df.head(2)


,date,store,item_id,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11


### date 컬럼을 dataframe의 index로 설정

In [6]:
data_df = data_df.set_index('date')
data_df.head(2)

,store,item_id,sales
date,,,
2013-01-01,1,1,13
2013-01-02,1,1,11


### 컬럼의 타입을 변경

- store: int --> String 
- item_id: int --> String

참고로 아래 ```train_df_store_astype(str)``` 로 변경시에 타입이 object 로 표시 됩니다. 이유는 string 이 변수 길이 (Variable Length)가 있어서, 디폴트로 object로 저장이 됩니다.

In [7]:
data_df.dtypes

store      int64
item_id    int64
sales      int64
dtype: object

In [8]:
data_df.store = data_df.store.astype(str)
data_df.item_id = data_df.item_id.astype(str)
data_df.dtypes

store      object
item_id    object
sales       int64
dtype: object

- index 의 date를 'object' --> datetime64 타입으로 변경

In [9]:
data_df.index[0:3]

Index(['2013-01-01', '2013-01-02', '2013-01-03'], dtype='object', name='date')

In [10]:
data_df.index = pd.to_datetime(data_df.index, format = '%Y-%m-%d')
data_df.index[0:3]

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03'], dtype='datetime64[ns]', name='date', freq=None)

## 최종 Forecast에 들어갈 입력 포맷

In [11]:
cols_order = ['item_id', 'store','sales']
data_df = data_df[cols_order]
print(data_df.index.name, data_df.index.dtype)
print(data_df.dtypes)


date datetime64[ns]
item_id    object
store      object
sales       int64
dtype: object


In [12]:
data_df.head() # 데이터 확인

,item_id,store,sales
date,,,
2013-01-01,1,1,13
2013-01-02,1,1,11
2013-01-03,1,1,14
2013-01-04,1,1,13
2013-01-05,1,1,10


## 사용할 데이터 기간 선택 

여기서는 추후 Predictor 및 Forecast를 에 사용할 데이터 기간을 정합니다. 현재 5년의 데이타를 다 사용하게 되면 훈련 시간이 오래 걸려서, 여기서는 최근 2년의 데이터만을 사용 합니다.
- 훈련 데이터 셋 (Train Data Set)
    - 데이타의 시작날짜, 미지막 날짜를 확인하고, 실제 학습에 사용할 사용할 기간을 2015-01-01 부터 2017-12-01 까지 23개월을 사용
- 검증 데어터 셋 (Validation Data Set)
    - 사용할 기간을 2017-12-01-2017-12-31일까지 1달을 사용.

In [13]:
print(data_df.index.max())
print(data_df.index.min())

2017-12-31 00:00:00
2013-01-01 00:00:00


### 경계 부분 날짜 정의

In [14]:
start_train_date = '2015-01-01' 
end_train_date = '2017-12-01' 
end_val_date = '2018-01-01' 

## train and validation datset 분리
- 데이타를 위의 두개의 데이타 셋으로 분리 하고, 확인 합니다.

In [15]:
stores_sales = data_df.copy()

In [16]:
# train_stores_sales: 2015.1.1 ~ 2016.11.30
# validation_stores_sales: 2016.12.01 ~ 2016.12.31
train_stores_sales = stores_sales[stores_sales.index >= start_train_date]
train_stores_sales = train_stores_sales[train_stores_sales.index < end_train_date]
validation_stores_sales = stores_sales[stores_sales.index >= end_train_date]
validation_stores_sales = validation_stores_sales[validation_stores_sales.index < end_val_date]

train 및 validation의 시작 및 끝 날짜를 확인 합니다.

In [17]:
train_stores_sales.head(2)

,item_id,store,sales
date,,,
2015-01-01,1,1,11
2015-01-02,1,1,19


In [18]:
train_stores_sales.tail(2)

,item_id,store,sales
date,,,
2017-11-29,50,10,76
2017-11-30,50,10,73


In [19]:
validation_stores_sales.head(2)

,item_id,store,sales
date,,,
2017-12-01,1,1,19
2017-12-02,1,1,16


In [20]:
validation_stores_sales.tail(2)

,item_id,store,sales
date,,,
2017-12-30,50,10,62
2017-12-31,50,10,82


## Train 및 Validation Data set을 CSV 파일로 저장

두 개의 데이타 셋을 두 개의 CSV 파일로 저장하여, 추후에 Forecast에서 Data Import시에 사용 합니다.
아래믄 먼저 CSV 파일로 저장할 Dataframe Row 개수를 확인 합니다.

In [21]:
train_stores_sales.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 532500 entries, 2015-01-01 to 2017-11-30
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   item_id  532500 non-null  object
 1   store    532500 non-null  object
 2   sales    532500 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 16.3+ MB


In [22]:
validation_stores_sales.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15500 entries, 2017-12-01 to 2017-12-31
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   item_id  15500 non-null  object
 1   store    15500 non-null  object
 2   sales    15500 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 484.4+ KB


In [23]:
train_time_series_filename = "train_target_time_series.csv"
train_time_series_path = data_dir + "/" + train_time_series_filename
train_stores_sales.to_csv(train_time_series_path, header=False)

In [24]:
validation_time_series_filename = "validation_time_series.csv"
validation_time_series_path = data_dir + "/" + validation_time_series_filename
validation_stores_sales.to_csv(validation_time_series_path, header=False)